In [1]:
import os
import glob
import re
import copy
import pandas as pd

def replace_str(target):
    target = target.replace('\n', '')
    target = target.replace(',', '')
    return target

def find_value(arr, target, jumpto=1):
    try:
        num = replace_str(arr[arr.index(target)+jumpto])
    except:
        raise ValueError(
            f'{arr}, {target}, {arr[arr.index(target)+jumpto]}')
        num = 'NA'
    return num

decode_time_pattern = re.compile("Decode: .*")
transform_pattern = re.compile("Transform .* END at_time")
process_time_pattern = re.compile("Load: .* Aug: .* End .*")
image_size_pattern = re.compile("image_size: .*")
logdir = "./log"
logdir_list = logdir.split('/')

output_file = f"decode*"
_logfile = logdir + "/" + output_file

logfiles = glob.glob(_logfile)

col_name = ["Decode time"]
parsed_dir = "./decode_parsed"
image_logs = {}

image_loglet = {}
image_size = ''
for logfile in logfiles:
    for line in open(logfile, 'r').readlines():
        if re.search(image_size_pattern, line) is not None:  # start log
            replace_txt = line.replace('\t', ' ')
            test = replace_txt.split(' ')
            info = list(filter(lambda x: x != "", test))
        
            image_size = find_value(info, "image_size:")
        elif re.search(decode_time_pattern, line) is not None:  # start log
#             print(line)
            replace_txt = line.replace('\t', ' ')
            test = replace_txt.split(' ')
            info = list(filter(lambda x: x != "", test))

            decode_time = find_value(info, "Decode:")
            image_loglet["Decode time"] = decode_time
            if image_size != '':
                if image_size in image_logs:
                    image_logs[image_size].append(image_loglet)
                else:
                    image_logs[image_size] = [ image_loglet ]
                image_loglet = {}

for image_size in image_logs:
    image_size_log = image_logs[image_size]
    image_df = pd.DataFrame(image_size_log,
                        columns=col_name)
    os.makedirs(f'{parsed_dir}',exist_ok=True)
    image_df.to_csv(f'{parsed_dir}/{image_size}.csv', sep=',')


In [2]:
dataset_col_name = ["Image size (KB)"] + col_name

dir_names = glob.glob(f'{parsed_dir}/*.csv')

total_log = []
for logdir in dir_names:
    file_name = logdir.split('/')[-1]
    image_size = file_name.replace('.csv','')

    df = pd.read_csv(
        f"./{parsed_dir}/{file_name}", index_col=None)
    df = df*1000
    total_loglet = {"Image size (KB)":image_size}
    
    for col in col_name:
        aug_df = df[col].dropna() 
        total_loglet[col] = f'{round(aug_df.mean(),2)}'
        
    total_log.append(total_loglet)

avg_df = pd.DataFrame(total_log,
                  columns=dataset_col_name)
avg_df.dropna().to_csv(f"./decode_total_summary.csv",
                   sep=',', na_rep='NA')
